In [21]:
import numpy as np
import pandas as pd
import os
import shutil
import glob
import matplotlib.pyplot as plt
import random
import torch
from torch import nn
import torchvision
import torch.utils.data
from torchvision.utils import save_image
import torch.optim as optim
from PIL import Image

from generative import main as generator
from solver import main as solver

In [32]:
def plot_images(x):
    plt.figure(figsize=(5,5))
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(x[i].squeeze())

def permute_image_pixel(image,permutation):
    c,h,w=image.shape
    image=image.reshape(-1,c)
    image=image[permutation,:]
    return image.reshape(c,h,w)

def permute_data(data,permutation):
    for i in range(data.shape[0]):
        data[i]=permute_image_pixel(data[i],permutation)
        
    return data

In [50]:
def get_dataset(permutation):
    data=pd.read_csv("./digit-recognizer/train.csv").values
    
    #seprate imgaes and labels
    x,y=data[:,1:],data[:,0]
    
    #reshape data to (-1,c,h,w)
    x=x.reshape((-1,1,28,28))
    
    #permute data
    x=permute_data(x,permutation)

    split_ratio=0.1
    total_data=x.shape[0]
    xtrain=x[int(total_data*split_ratio):]
    ytrain=y[int(total_data*split_ratio):]
    xtest=x[:int(total_data*split_ratio)]
    ytest=y[:int(total_data*split_ratio)]
    return xtrain,ytrain,xtest,ytest

In [51]:
permutation=np.random.RandomState(seed=i).permutation(28*28)
xtrain,ytrain,xtest,ytest=get_dataset(permutation)
print(xtrain.shape,ytrain.shape,xtest.shape,ytest.shape)

(37800, 1, 28, 28) (37800,) (4200, 1, 28, 28) (4200,)


In [ ]:
for i in range(1):
    
    permutation=np.random.RandomState(seed=i).permutation(28*28)
    xtrain,ytrain,xtest,ytest=get_dataset(permutation)
    
    dis="./generators/gen.pt"
    gen="./generators/dis.pt"
    
    if(i>0): # ignore for the first task
        
    